# 🧩 Imports & Setup
This cell imports the required libraries and sets up the environment for our debate app.

In [ ]:
import gradio as gr
import ollama
import time

# ⚙️ System Prompts (Personalities)
We define two distinct Tunisian chatbot personalities — an enthusiastic defender and a calm diplomat.

In [ ]:
CHATBOT1_SYSTEM = (
    "You are a passionate Tunisian enthusiast, ya weldi! "
    "You love kuskus and defend it fiercely with a fiery Tunisian vibe. "
    "Use English with Tunisian expressions. Be emotional and proud."
)

CHATBOT2_SYSTEM = (
    "You are a polite Tunisian diplomat, calm and respectful. "
    "You love brik and always try to find a middle ground in debates. "
    "Use English with Tunisian flavor. Stay friendly and balanced."
)

# 💬 Global State
We keep track of each chatbot’s messages to simulate a continuous debate flow.

In [ ]:
chatbot1_messages = []
chatbot2_messages = []

# 🔄 Streaming Function
This function connects to Ollama’s API and streams chatbot replies chunk by chunk to create a live debate effect.

In [ ]:
def ollama_stream(model, messages):
    """Stream text chunks progressively from the model."""
    stream = ollama.chat(model=model, messages=messages, stream=True)
    text = ""
    for chunk in stream:
        content = chunk["message"]["content"]
        text += content
        yield text
        time.sleep(0.05)

# 🧠 Chatbot Interaction Logic
Each chatbot uses its previous messages and the opponent’s messages to build context before generating new replies.

In [ ]:
def call_chatbot1():
    messages = [{"role": "system", "content": CHATBOT1_SYSTEM}]
    for c1, c2 in zip(chatbot1_messages, chatbot2_messages):
        messages.append({"role": "assistant", "content": c1})
        messages.append({"role": "user", "content": c2})
    return ollama_stream("llama3.2", messages)

def call_chatbot2():
    messages = [{"role": "system", "content": CHATBOT2_SYSTEM}]
    for c1, c2 in zip(chatbot1_messages, chatbot2_messages):
        messages.append({"role": "user", "content": c1})
        messages.append({"role": "assistant", "content": c2})
    messages.append({"role": "user", "content": chatbot1_messages[-1]})
    return ollama_stream("llama3.2", messages)

# 🥊 Debate Function (Automatic Version)
This function runs a 3-round debate based on a topic (like “Kuskus vs. Brik”) and displays each chatbot’s responses as they stream.

In [ ]:
def run_debate(topic):
    global chatbot1_messages, chatbot2_messages
    chatbot1_messages, chatbot2_messages = [], []

    try:
        item1, item2 = topic.split(" vs. ")
    except ValueError:
        yield "⚠️ Please enter topic in format: `Kuskus vs. Brik`"
        return

    chatbot1_messages = [f"Ya weldi, {item1.strip()} is the pride of Tunisia! Nothing beats it!"]
    chatbot2_messages = [f"Wallahi, {item2.strip()} is special too, ya weldi. Let's stay calm."]

    conversation_log = f"🔥 **Debate Topic:** {topic}\n\n"
    conversation_log += f"**Chatbot 1 (Enthusiast):** {chatbot1_messages[0]}\n\n"
    conversation_log += f"**Chatbot 2 (Diplomat):** {chatbot2_messages[0]}\n\n"
    yield conversation_log

    # --- 3 Rounds ---
    for round in range(3):
        conversation_log += f"--- **Round {round+1}** ---\n\n"

        # Enthusiast reply
        conversation_log += f"**Chatbot 1 (Enthusiast):** "
        response1 = ""
        for chunk in call_chatbot1():
            response1 = chunk
            temp_log = conversation_log + response1
            yield temp_log
        chatbot1_messages.append(response1)
        conversation_log += response1 + "\n\n"

        # Diplomat reply
        conversation_log += f"**Chatbot 2 (Diplomat):** "
        response2 = ""
        for chunk in call_chatbot2():
            response2 = chunk
            temp_log = conversation_log + response2
            yield temp_log
        chatbot2_messages.append(response2)
        conversation_log += response2 + "\n\n"

    yield conversation_log

# 💬 Conversation Mode (Free Topic)
Here, you can give the chatbots any topic or question — they’ll chat about it freely without a strict “vs.” format.

In [ ]:
def free_conversation(topic):
    global chatbot1_messages, chatbot2_messages
    chatbot1_messages, chatbot2_messages = [], []

    chatbot1_messages = [f"Ya weldi, I heard about {topic.strip()}! It's so interesting, sah?"]
    chatbot2_messages = [f"Indeed, ya weldi, {topic.strip()} is worth discussing. Let's share ideas calmly."]

    conversation_log = f"💡 **Conversation Topic:** {topic}\n\n"
    conversation_log += f"**Chatbot 1:** {chatbot1_messages[0]}\n\n"
    conversation_log += f"**Chatbot 2:** {chatbot2_messages[0]}\n\n"
    yield conversation_log

    # --- 3 Rounds ---
    for round in range(3):
        conversation_log += f"--- **Round {round+1}** ---\n\n"

        # Enthusiast
        conversation_log += f"**Chatbot 1:** "
        response1 = ""
        for chunk in call_chatbot1():
            response1 = chunk
            yield conversation_log + response1
        chatbot1_messages.append(response1)
        conversation_log += response1 + "\n\n"

        # Diplomat
        conversation_log += f"**Chatbot 2:** "
        response2 = ""
        for chunk in call_chatbot2():
            response2 = chunk
            yield conversation_log + response2
        chatbot2_messages.append(response2)
        conversation_log += response2 + "\n\n"

    yield conversation_log

# 🎨 Gradio Interface
We design a beautiful Gradio interface with two tabs: One for Debate Mode and one for Free Conversation Mode.

In [ ]:
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown(
        """
        <h1 style='text-align:center;'>🇹🇳 Tunisian Debate 🇹🇳</h1>
        <p style='text-align:center;'>Watch two Tunisian AIs argue or discuss — proudly, emotionally, and humorously 💬</p>
        """
    )

    with gr.Tab("🔥 Debate Mode"):
        topic_input = gr.Textbox(label="Enter Debate Topic (e.g., Kuskus vs. Brik)", value="Kuskus vs. Brik")
        start_button = gr.Button("Start Debate")
        output = gr.Markdown(label="Debate Transcript")
        start_button.click(run_debate, inputs=topic_input, outputs=output)

    with gr.Tab("💡 Free Conversation"):
        convo_input = gr.Textbox(label="Enter a Topic (e.g., Tunisian Food Culture)", value="Tunisian Food Culture")
        start_convo = gr.Button("Start Chat")
        convo_output = gr.Markdown(label="Conversation Transcript")
        start_convo.click(free_conversation, inputs=convo_input, outputs=convo_output)

# 🚀 Launch the App
Finally, we launch the Gradio interface to start our Tunisian AI showdown.

In [ ]:
if __name__ == "__main__":
    app.launch()